In [5]:
import os
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from matplotlib import pyplot as plt
from torchvision.utils import save_image, make_grid

In [6]:
def logging_image_grid(images, captions, path, ncol=7, normalize = True, save = True):
    if not normalize:
        norm_text = "_not_normalized"
    else:
        norm_text = ""

    grids = [make_grid(img, nrow=ncol,padding=1, normalize=normalize, scale_each=True) for img in images]
    for grid, caption in zip(grids, captions):
        if save:
            save_image(grid, path +  '/' + caption + norm_text + '.png')
        else:
            plt.imshow(np.asarray(grid.permute((1,2,0)).cpu()[:,:,0]), cmap='plasma')
            plt.title(caption)
            plt.axis('off')
            plt.show()
    return

In [9]:
gt_path = "/home/samanta/touch2touch/results/ground_truth"
models_path = "/home/samanta/depth_anything_v2/metric_depth/results"
models_paths = ['fine_tuning_bubbles_max0.12_no_mask_not_zero_train_tools_improved_data']
models_paths = [os.path.join(models_path, path) for path in models_paths]

### Visual Qualitative Results

In [11]:
for i, model_path in enumerate(models_paths):
    model = model_path.split('/')[-1]
    print(model)
    datasets_paths = os.listdir(model_path)
    datasets_paths = [path for path in datasets_paths if os.path.isdir(os.path.join(model_path, path))]
    datasets_paths.sort()
    for dataset in datasets_paths:
        inputs = []
        gt = []
        pred = []
        dataset_path = os.path.join(model_path, dataset)
        datasets_paths = ['test_unseen']
        print(dataset)
        tools_paths = os.listdir(dataset_path)
        tools_paths.sort()
        tools_paths = tools_paths[:4]
        for tool in tools_paths:
            tool_path = os.path.join(dataset_path, tool)
            tool_input = torch.load(os.path.join(tool_path, "depth_qualitative_results.pt"))['bubbles_img_viz_single']
            tool_gt = torch.load(os.path.join(tool_path, "depth_qualitative_results.pt"))['depth_gt_viz_single']
            tool_pred = torch.load(os.path.join(tool_path, "depth_qualitative_results.pt"))['depth_pred_viz_single']
            inputs.append(tool_input)
            gt.append(tool_gt)
            pred.append(tool_pred)

        inputs = torch.cat(inputs, dim=0)
        gt = torch.cat(gt, dim=0)
        pred = torch.cat(pred, dim=0)
        gt = torch.cat([inputs, gt, pred], dim=0)
        

        if i == 0:
            logging_image_grid([inputs, gt], [f"Input_{dataset}", f"GT_{dataset}", f"GT_PCD_{dataset}"], '',save=False, ncol=4)
        logging_image_grid([pred], [f"Output_{dataset}", f"Output_PCD_{model}_{dataset}"], "", save=False, ncol=4)

fine_tuning_bubbles_max0.12_no_mask_not_zero_train_tools_improved_data
test


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 3 but got size 1 for tensor number 1 in the list.

### Visual Quantitative Results

In [25]:
metrics_results = []

for i, model_path in enumerate(models_paths):
    model = model_path.split('/')[-1]
    datasets_paths = ['test_unseen']
    datasets_paths = [path for path in datasets_paths if os.path.isdir(os.path.join(model_path, path))]
    datasets_paths.sort()
    
    for dataset in datasets_paths:
        abs_rel_error = 0
        rmse_error = 0
        log_rmse_error = 0
        silog_error = 0

        dataset_path = os.path.join(model_path, dataset)
        tools_paths = os.listdir(dataset_path)
        tools_paths.sort()
        # tools_paths = tools_paths[:4]

        for tool in tools_paths:
            tool_path = os.path.join(dataset_path, tool)
            metrics = torch.load(os.path.join(tool_path, "depth_quantitative_results.pt"))

            abs_rel_error += metrics['abs_rel']
            rmse_error += metrics['rmse']
            log_rmse_error += metrics['log_rmse']
            silog_error += metrics['silog']

        # Compute average errors over all tools
        abs_rel_error /= len(tools_paths)
        rmse_error /= len(tools_paths)
        log_rmse_error /= len(tools_paths)
        silog_error /= len(tools_paths)
        
        # Store results in dictionary
        metrics_dict = {
            'method': model, 
            'dataset': dataset, 
            'AbsRel': abs_rel_error, 
            'RMSE': rmse_error, 
            'LogRMSE': log_rmse_error, 
            'SiLog': silog_error
        }
        
        metrics_results.append(metrics_dict)

# Convert to DataFrame
metrics_df = pd.DataFrame(metrics_results)

# Print results for different datasets
for dataset_name in ['train', 'test', 'test_unseen']:
    filtered_df = metrics_df[metrics_df['dataset'] == dataset_name]
    pd.set_option('display.precision', 4)  # Set precision for better readability
    print(f'{dataset_name.capitalize()} Results:')
    print(filtered_df)
    print('\n' + '-'*50 + '\n')

Train Results:
Empty DataFrame
Columns: [method, dataset, AbsRel, RMSE, LogRMSE, SiLog]
Index: []

--------------------------------------------------

Test Results:
Empty DataFrame
Columns: [method, dataset, AbsRel, RMSE, LogRMSE, SiLog]
Index: []

--------------------------------------------------

Test_unseen Results:
                                              method      dataset  AbsRel  \
0  fine_tuning_bubbles_max0.12_no_mask_not_zero_t...  test_unseen  0.1135   

     RMSE  LogRMSE   SiLog  
0  0.0236   3.3096  2.7207  

--------------------------------------------------

